In [46]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pycountry_convert as pc
import seaborn as sns
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
df= pd.read_csv('Earthquake_with_continents.csv')

df
print(df.isnull().sum())
df['date']=pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df.head()

Unnamed: 0            0
id                    0
magnitude             0
type                  0
title                 0
date                  0
time                  0
updated               0
url                   0
detailUrl             0
felt                  0
cdi                   0
mmi                   0
alert              4188
status                0
tsunami               0
sig                   0
net                   0
code                  0
ids                   0
sources               0
types                 0
nst                   0
dmin                  0
rms                   0
gap                   0
magType               0
geometryType          0
depth                 0
latitude              0
longitude             0
place                89
distanceKM           55
placeOnly            89
location             89
continent             0
country             949
subnational        1043
city               3855
locality              4
postcode           2503
what3words      

,Unnamed: 0,id,magnitude,type,title,date,time,updated,url,detailUrl,...,continent,country,subnational,city,locality,postcode,what3words,timezone,locationDetails,month
0,0,at00rt29jk,6.2,earthquake,"M 6.2 - 211 km SW of Port McNeill, Canada",2023-04-13 15:54:56,1681401296000,1681402393396,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,...,Ocean/Unknown,NaN,NaN,NaN,Etc/GMT+9,NaN,tinkering.retail.exiting,-540,"[{'id': '86035', 'wikidataId': 'Q12353254', 'n...",4
1,1,at00rt29jk,6.2,earthquake,"M 6.2 - 211 km SW of Port McNeill, Canada",2023-04-13 15:54:56,1681401296000,1681402393396,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,...,Ocean/Unknown,NaN,NaN,NaN,Etc/GMT+9,NaN,tinkering.retail.exiting,-540,"[{'id': '86035', 'wikidataId': 'Q12353254', 'n...",4
2,2,us6000k42y,6.0,earthquake,"M 6.0 - 238 km SW of Port McNeill, Canada",2023-04-13 15:54:53,1681401293815,1681481900040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,...,Ocean/Unknown,NaN,NaN,NaN,Etc/GMT+9,NaN,multipliers.locating.gushing,-540,"[{'id': '86035', 'wikidataId': 'Q12353254', 'n...",4
3,3,us6000k3z2,5.4,earthquake,M 5.4 - Owen Fracture Zone region,2023-04-13 03:24:08,1681356248627,1681433567440,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,...,Ocean/Unknown,NaN,NaN,NaN,Etc/GMT-4,NaN,beams.inactivation.prepacked,240,"[{'id': '621', 'wikidataId': 'Q58705', 'name':...",4
4,4,us6000k3z1,5.2,earthquake,"M 5.2 - Nicobar Islands, India region",2023-04-13 03:21:19,1681356079365,1681432280040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,...,Ocean/Unknown,NaN,NaN,NaN,Etc/GMT-6,NaN,ejected.attainments.peruse,360,"[{'id': '428', 'wikidataId': 'Q47632', 'name':...",4


In [54]:
#This is not showing on my notebook?
sns.relplot(data=df,x="depth", y="nst", hue='continent', palette = "bright",col='month')

In [48]:
#Showing the realtionship between the depth of the activity and the nst with the 
#Continent as the colours.
fig = px.scatter(df, x="depth", y="nst", color='continent', size= 'magnitude')
fig.show()

In [45]:
#This graph isn't showing for me?
sns.jointplot(data=df, x= 'magnitude', y='mmi',hue='tsunami',sizes='felt' )

In [43]:
#Interactive plot showing the number of each quake per month.
df.sort_values('month',inplace=True)
fig = px.histogram(df, y="type", animation_frame="month",
            range_x=[0,200], color='type')

fig.show()

In [72]:
# Showing the location of each of the activities. 
#Gonna go ahead and only plot the quakes that have a magnitude of over 4
import folium

# Create a map of the USA
m = folium.Map(tiles="Stamen Terrain")

df2=df[df.magnitude>=4.0]

# Add markers for the airports
for i in range(len(df2)):
    lon = df2.longitude[i]
    lat = df2.latitude[i]
    name = df2.type[i]
    mag=df2.magnitude[i]
    for j in range(len(df2.type.unique())):
        cols=['blue','red','green','purple']
        if name==df.type.unique()[j]:
            folium.Marker([lat, lon], popup=[name,mag],icon=folium.Icon(color=cols[j])).add_to(m)

In [81]:
# define the buttons to zoom into each continent
buttons = [
    {'name': 'Africa', 'location': [0, 20], 'zoom': 4},
    {'name': 'Asia', 'location': [30, 100], 'zoom': 4},
    {'name': 'Europe', 'location': [50, 10], 'zoom': 4},
    {'name': 'North America', 'location': [40, -100], 'zoom': 3},
    {'name': 'South America', 'location': [-15, -60], 'zoom': 4},
    {'name': 'Australia', 'location': [-25, 135], 'zoom': 4},
]

button_template = """
    <button onclick="myFunction('{name}', {lat}, {lng}, {zoom})">Zoom to {name}</button>
"""
button_layer = folium.FeatureGroup(name='Continents')

# Add the buttons to the button layer
for button in buttons:
    html = button_template.format(name=button['name'], lat=button['location'][0], lng=button['location'][1], zoom=button['zoom'])
    iframe = folium.IFrame(html=html, width=200, height=50)
    popup = folium.Popup(iframe, max_width=2650)
    marker = folium.Marker(location=button['location'], popup=popup)
    marker.add_to(button_layer)

# Add the button layer to the map
button_layer.add_to(m)

# Add a click listener to the map that zooms to the continent
callback = """
    function myFunction(name, lat, lng, zoom) {
        var map = window.map;
        map.setView([lat, lng], zoom);
    };
"""
folium.GeoJson(
    data={'type': 'FeatureCollection', 'features': []},
    style_function=lambda feature: {'fillColor': 'white'},
    highlight_function=lambda feature: {'weight': 3},
    control=False,
    smooth_factor=0,
    tooltip=None,
    name='Click to zoom',
    show=False,
).add_to(m)
m.get_root().add_child(folium.Element(callback))

# Add layer control
folium.LayerControl().add_to(m)

In [82]:
m

In [83]:
# If we were to do this with all the data, the image is extremely laggy
# And the only place that shows anything other than earthquakes
# Is america. 
#I have no idea how to make buttons that zoom... 